In [1]:
import re
import os
from langchain.vectorstores.neo4j_vector import Neo4jVector
from langchain.document_loaders import WikipediaLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import (
    CharacterTextSplitter,
    RecursiveCharacterTextSplitter,
)
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from langchain.text_splitter import TokenTextSplitter
from langchain.document_loaders import DiffbotLoader
from secret import *

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [2]:
import logging
from urllib.parse import urljoin
import requests
from bs4 import BeautifulSoup

search_query = "atelectasis"
URL = f"https://radiopaedia.org/search?lang=gb&q={search_query}&scope=articles"
page = requests.get(URL)
soup = BeautifulSoup(page.content, "html.parser")
radio_address = "https://radiopaedia.org"
all_address = [
    f"{radio_address}{e['href']}"
    for e in soup.find_all("a", class_="search-result search-result-article")
]
all_address

['https://radiopaedia.org/articles/atelectasis-summary?lang=gb',
 'https://radiopaedia.org/articles/linear-atelectasis?lang=gb',
 'https://radiopaedia.org/articles/lung-atelectasis?lang=gb',
 'https://radiopaedia.org/articles/subsegmental-atelectasis?lang=gb',
 'https://radiopaedia.org/articles/round-atelectasis?lang=gb',
 'https://radiopaedia.org/articles/passive-atelectasis-1?lang=gb',
 'https://radiopaedia.org/articles/compressive-atelectasis?lang=gb',
 'https://radiopaedia.org/articles/cicatrisation-atelectasis?lang=gb',
 'https://radiopaedia.org/articles/segmental-atelectasis?lang=gb',
 'https://radiopaedia.org/articles/adhesive-atelectasis?lang=gb',
 'https://radiopaedia.org/articles/resorptive-obstructive-atelectasis?lang=gb',
 'https://radiopaedia.org/articles/chronic-maxillary-atelectasis?lang=gb',
 'https://radiopaedia.org/articles/middle-ear-atelectasis?lang=gb',
 'https://radiopaedia.org/articles/gravity-dependent-atelectasis-1?lang=gb',
 'https://radiopaedia.org/articles/c

In [3]:
raw_documents = DiffbotLoader(
    urls=all_address,
    api_token=DIFFBOT_API_TOKEN,
).load()
text_splitter = TokenTextSplitter(chunk_size=2048, chunk_overlap=24)
documents = text_splitter.split_documents(
    raw_documents
)  # should I only take the first? which is the most relevant?

In [4]:
from langchain.indexes import GraphIndexCreator
from langchain.graphs.networkx_graph import KnowledgeTriple
from langchain.llms import OpenAI

In [5]:
index_creator = GraphIndexCreator(llm=OpenAI(temperature=0))

In [6]:
graph = None
for d in documents:
    g_temp = index_creator.from_text(d.page_content)
    if graph is None:
        graph = g_temp
    else:
        triplets = graph.get_triples()
        for t in triplets:
            graph.add_triple(knowledge_triple=KnowledgeTriple(*t))

In [7]:
from langchain.chains import GraphQAChain

In [8]:
chain = GraphQAChain.from_llm(OpenAI(temperature=0), graph=graph, verbose=True)

In [9]:
chain.run("What is Atelectasis?")



> Entering new GraphQAChain chain...
Entities Extracted:
 Atelectasis
Full Context:
Atelectasis describes small areas of collapsed lung
Atelectasis is caused by adjacent compression
Atelectasis is caused by passive atelectasis
Atelectasis is caused by dependent atelectasis
Atelectasis is usually seen on chest x-rays as small volume linear shadows
Atelectasis is often helpful to determine the cause or confirm that there is no proximal obstruction CT is
Atelectasis small areas of collapsed lung describes
Atelectasis dependent atelectasis is caused by
Atelectasis small volume linear shadows is usually seen on chest x-rays as
Atelectasis CT is is often helpful to determine the cause or confirm that there is no proximal obstruction

> Finished chain.


' Atelectasis is a condition in which small areas of the lung collapse due to adjacent compression, passive atelectasis, or dependent atelectasis. It is usually seen on chest x-rays as small volume linear shadows and CT is often helpful to determine the cause or confirm that there is no proximal obstruction.'

In [10]:
chain.run("What are the symptoms of Atelectasis?")



> Entering new GraphQAChain chain...
Entities Extracted:
 Atelectasis
Full Context:
Atelectasis describes small areas of collapsed lung
Atelectasis is caused by adjacent compression
Atelectasis is caused by passive atelectasis
Atelectasis is caused by dependent atelectasis
Atelectasis is usually seen on chest x-rays as small volume linear shadows
Atelectasis is often helpful to determine the cause or confirm that there is no proximal obstruction CT is
Atelectasis small areas of collapsed lung describes
Atelectasis dependent atelectasis is caused by
Atelectasis small volume linear shadows is usually seen on chest x-rays as
Atelectasis CT is is often helpful to determine the cause or confirm that there is no proximal obstruction

> Finished chain.


' Atelectasis is usually asymptomatic, but some people may experience shortness of breath, chest pain, and coughing.'

In [11]:
chain.run("What can cause Atelectasis?")



> Entering new GraphQAChain chain...
Entities Extracted:
 Atelectasis
Full Context:
Atelectasis describes small areas of collapsed lung
Atelectasis is caused by adjacent compression
Atelectasis is caused by passive atelectasis
Atelectasis is caused by dependent atelectasis
Atelectasis is usually seen on chest x-rays as small volume linear shadows
Atelectasis is often helpful to determine the cause or confirm that there is no proximal obstruction CT is
Atelectasis small areas of collapsed lung describes
Atelectasis dependent atelectasis is caused by
Atelectasis small volume linear shadows is usually seen on chest x-rays as
Atelectasis CT is is often helpful to determine the cause or confirm that there is no proximal obstruction

> Finished chain.


' Atelectasis can be caused by adjacent compression, passive atelectasis, and dependent atelectasis.'

In [12]:
chain.run("What clinical features radiologists need for diagnosing Atelectasis?")



> Entering new GraphQAChain chain...
Entities Extracted:
 Atelectasis
Full Context:
Atelectasis describes small areas of collapsed lung
Atelectasis is caused by adjacent compression
Atelectasis is caused by passive atelectasis
Atelectasis is caused by dependent atelectasis
Atelectasis is usually seen on chest x-rays as small volume linear shadows
Atelectasis is often helpful to determine the cause or confirm that there is no proximal obstruction CT is
Atelectasis small areas of collapsed lung describes
Atelectasis dependent atelectasis is caused by
Atelectasis small volume linear shadows is usually seen on chest x-rays as
Atelectasis CT is is often helpful to determine the cause or confirm that there is no proximal obstruction

> Finished chain.


' Radiologists need to look for small areas of collapsed lung, small volume linear shadows, and adjacent compression on chest x-rays to diagnose Atelectasis. CT scans may also be helpful to determine the cause or confirm that there is no proximal obstruction.'

In [13]:
chain.run("what are radiographic features of Atelectasis?")



> Entering new GraphQAChain chain...
Entities Extracted:
 Atelectasis
Full Context:
Atelectasis describes small areas of collapsed lung
Atelectasis is caused by adjacent compression
Atelectasis is caused by passive atelectasis
Atelectasis is caused by dependent atelectasis
Atelectasis is usually seen on chest x-rays as small volume linear shadows
Atelectasis is often helpful to determine the cause or confirm that there is no proximal obstruction CT is
Atelectasis small areas of collapsed lung describes
Atelectasis dependent atelectasis is caused by
Atelectasis small volume linear shadows is usually seen on chest x-rays as
Atelectasis CT is is often helpful to determine the cause or confirm that there is no proximal obstruction

> Finished chain.


' Atelectasis is usually seen on chest x-rays as small volume linear shadows.'